Name - Aastha Kundra

PGID - 12310022

### Part 4: Image Classification using Transfer Learning

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
print(os.getcwd())
os.chdir('C:\\Users\\aasth\\Downloads')

C:\Users\aasth\Downloads


#### 1.Download the dataset from: https://www.kaggle.com/datasets/kmkarakaya/logos-bk-kfc-mcdonald-starbucks-subway-none

In [2]:
# Load the dataset
data_dir = "./logos_v3_mini/logos3"
print(sorted(os.listdir(data_dir)))

train_path = data_dir+'/train'
test_path  = data_dir+'/test'


class_names = sorted(os.listdir(train_path))
print(sorted(os.listdir(train_path)))
print(sorted(os.listdir(test_path)))

['test', 'train']
['Burger King', 'KFC', 'McDonalds', 'Other', 'Starbucks', 'Subway']
['Burger King', 'KFC', 'McDonalds', 'Other', 'Starbucks', 'Subway']


#### 2.Preprocess the data by resizing the images and splitting them into training (80%) and validation sets (20%).

In [3]:
# Data preprocessing and splitting
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation')

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 1393 images belonging to 6 classes.
Found 345 images belonging to 6 classes.
Found 560 images belonging to 6 classes.


In [4]:
selectedClasses =  sorted(os.listdir(train_path))

print ("In train_generator ")
for cls in range(len (train_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(train_generator.classes).count(cls))
print ("") 

print ("In validation_generator ")
for cls in range(len (validation_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(validation_generator.classes).count(cls))
print ("") 

print ("In test_generator ")
for cls in range(len (test_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(test_generator.classes).count(cls))

In train_generator 
Burger King :	 238
KFC :	 56
McDonalds :	 152
Other :	 660
Starbucks :	 187
Subway :	 100

In validation_generator 
Burger King :	 59
KFC :	 14
McDonalds :	 37
Other :	 165
Starbucks :	 46
Subway :	 24

In test_generator 
Burger King :	 153
KFC :	 20
McDonalds :	 96
Other :	 224
Starbucks :	 38
Subway :	 29


#### 3.Load a pre-trained model, such as VGG-16 or ResNet, using Keras or PyTorch. (or a library of your choice)

In [5]:
base_model = VGG16(weights='imagenet', include_top=False)

#### 4. Replace the last fully connected layer of the pre-trained model with a new fully connected layer

In [6]:
# Add custom fully connected layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

#### 5. Freeze the weights of the pre-trained layers and train only the new fully connected layer using the training set.

In [7]:
# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 15,246,150 (58.16 MB)

 Trainable params: 531,462 (2.03 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [8]:
from tensorflow.keras.utils import Sequence

class CustomDataGenerator(Sequence):
    def __init__(self, generator):
        self.generator = generator

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, index):
        return self.generator[index]

train_data_generator = CustomDataGenerator(train_generator)
validation_data_generator = CustomDataGenerator(validation_generator)

history = model.fit(
    train_data_generator,
    steps_per_epoch=len(train_data_generator),
    validation_data=validation_data_generator,
    validation_steps=len(validation_data_generator),
    epochs=5
)


Epoch 1/5


C:\Users\aasth\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/44 ━━━━━━━━━━━━━━━━━━━━ 260s 6s/step - accuracy: 0.5791 - loss: 1.1392 - val_accuracy: 0.7768 - val_loss: 0.6560
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/5


C:\Users\aasth\anaconda3\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


44/44 ━━━━━━━━━━━━━━━━━━━━ 259s 6s/step - accuracy: 0.8297 - loss: 0.4811 - val_accuracy: 0.8435 - val_loss: 0.4791
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 342s 8s/step - accuracy: 0.9310 - loss: 0.2995 - val_accuracy: 0.8783 - val_loss: 0.3819


#### 6.Evaluate the performance of the model using the validation set.

In [9]:
loss, accuracy = model.evaluate(validation_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

11/11 ━━━━━━━━━━━━━━━━━━━━ 54s 5s/step - accuracy: 0.8777 - loss: 0.3959
Validation Loss: 0.38189223408699036
Validation Accuracy: 0.8782608509063721


#### 7. Fine-tune the entire model by unfreezing some of the pre-trained layers

In [10]:
# Unfreeze some pre-trained layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5)


Epoch 1/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 285s 7s/step - accuracy: 0.8856 - loss: 0.3437 - val_accuracy: 0.9406 - val_loss: 0.1726
Epoch 2/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.9688 - loss: 0.1243 - val_accuracy: 0.9600 - val_loss: 0.1706
Epoch 3/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 297s 7s/step - accuracy: 0.9948 - loss: 0.0223 - val_accuracy: 0.9563 - val_loss: 0.0955
Epoch 4/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 9s 93ms/step - accuracy: 0.9688 - loss: 0.0488 - val_accuracy: 0.9600 - val_loss: 0.0783
Epoch 5/5
43/43 ━━━━━━━━━━━━━━━━━━━━ 303s 7s/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 0.9625 - val_loss: 0.0918


#### 8. Evaluate the performance of the model using the validation set.

In [12]:
loss_fine_tune, accuracy_fine_tune = model.evaluate(validation_generator)
print("Validation Loss (Fine-tuned):", loss_fine_tune)
print("Validation Accuracy (Fine-tuned):", accuracy_fine_tune)


11/11 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - accuracy: 0.9799 - loss: 0.0533
Validation Loss (Fine-tuned): 0.08531782776117325
Validation Accuracy (Fine-tuned): 0.9652174115180969


#### 9. A report summarizing the findings, including the best performing model and the factors that contributed to its success.

Factors Contributing to Success: 

-Unfreezing and fine-tuning the pre-trained layers allowed the model to learn more specific features from the new dataset, resulting in improved performance. The fine-tuning process enabled the model to capture higher-level representations relevant to the food chain logos, leading to better classification accuracy. 

-The improvement in accuracy demonstrates the effectiveness of transfer learning in leveraging pre-trained models for tasks with limited training data.

Overall, unfreezing and fine-tuning the pre-trained layers significantly enhanced the model's performance, highlighting the importance of adapting pre-trained features to the target dataset for better classification accuracy.
